In [1]:
import pandas as pd

In [15]:
df = pd.read_excel('logs.xlsx', sheet_name="Tabelle1")

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13515 entries, 0 to 13514
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Besitzer Name_2 (TPL)            13515 non-null  object        
 1   Fz-Familie (TPL)                 13515 non-null  object        
 2   EFA Nr (MEL)                     13515 non-null  object        
 3   DIN-Code UPG (H)                 13515 non-null  object        
 4   Baugruppe (DIN-Code) (MEL)       13515 non-null  object        
 5   Fz-Typ (TPL)                     13515 non-null  object        
 6   Triebzug / EW (TPL)              13515 non-null  object        
 7   Technischer Platz (TPL)          13515 non-null  object        
 8   Störungsbeginn Datum (MEL)       13515 non-null  datetime64[ns]
 9   Kurztext (MEL)                   13515 non-null  object        
 10  Shorttext(en)                    0 non-null      float64  